Mount Google Drive

In [115]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Imports/ResNet Class

In [116]:
# import packages
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, ZeroPadding1D, Conv1D, AveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import Activation, Dense,  Flatten, Input, add, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow import summary
from sklearn.model_selection import train_test_split
import numpy as np



#define resnet class
class ResNet:
  @staticmethod
  #create one residual block
  def residual_module(data, K, convLayers, chanDim=-1, reduce=False, reg=0.0001, bnEps=2e-5, bnMom=0.9, dropout=0.8, kernel_size=1):
    K = K*0.25
    shortcut = data
    x = data

    print("Resnet Block")
    for i in range(convLayers):
      x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
      x = Activation("relu")(x)
      
      if (i == convLayers - 1):
        K = K*4
        x = Conv1D(int(K), kernel_size, use_bias=False, kernel_regularizer=l2(reg))(x)
      elif (i == int(convLayers/2)):
        x = Conv1D(int(K), kernel_size, use_bias=False, kernel_regularizer=l2(reg))(x)
      else:
        x = Conv1D(int(K), kernel_size, use_bias=False, kernel_regularizer=l2(reg))(x)
      if (dropout is not None):
        x = Dropout(dropout)(x)
     
      #print("**PostConvolution", x.shape)
      print("*ConvGroup", x.shape, K)

    if reduce:
      #shortcut = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(shortcut)
      shortcut = Conv1D(K, kernel_size, use_bias=False, kernel_regularizer=l2(reg))(shortcut)
    print("*Shortcut shape", shortcut.shape)

    x = add([x, shortcut])

    return x

  @staticmethod
  #build a model out of resblocks
  def build(inputShape, resLayers, classes, convLayers, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9, dropout=None, kernel_size=1):
    chanDim = -1
    inputs = Input(shape=inputShape)

    #initial batchnorm and convolutional layer
    x = BatchNormalization(axis=chanDim, epsilon = bnEps, momentum=bnMom)(inputs)
    x = Conv1D(filters[0], 5, strides=2, use_bias=False, padding="same", kernel_regularizer=l2(reg))(inputs)
    
    x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
    x = Activation("relu")(x)

       
    x = ResNet.residual_module(data=x, K=filters[1], convLayers=convLayers, reduce=True, bnEps=bnEps, bnMom=bnMom)

    for i in range(0, resLayers - 1):
      x = ResNet.residual_module(data=x, K=filters[1], convLayers=convLayers, bnEps=bnEps, bnMom=bnMom)
    
    x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = AveragePooling1D(8)(x)
    x = Flatten()(x)
    x = Dense(classes, kernel_regularizer=l2(reg))(x)
    x = Activation("softmax")(x)

    model = Model(inputs, x, name="resnet")

    return model

  @staticmethod
  def pretrain(save_directory, features, labels, verbose=1,
                                   batch_size=32, 
                                   epochs=10, 
                                   filters=(64, 100), 
                                   resLayers=1, 
                                   convLayers=1, 
                                   learningRate=0.1,
                                   dropout=0.8,
                                   kernel_size=1):
    X = features
    y = labels

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)


    #add depth channel
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    #input shape is everything except for the number of samples 
    #number of classes is the number of unique items in y
    in_shape = X_train.shape[1:]
    n_classes = len(np.unique(y_train))

    #build model
    model = ResNet.build(inputShape=in_shape, classes=n_classes, convLayers=convLayers, resLayers=resLayers, filters=filters, dropout=dropout)

    #compile, train model
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer=Adam(learningRate), metrics = ['accuracy'])
    
    model.summary()
    
    model.fit(X_train, y_train, batch_size=batch_size,epochs=epochs, verbose=verbose)

    #test accuracy
    accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    print("(pretraining) Testing accuracy from within X_reference.npy and y_reference.npy:", accuracy[1])

    # Save the weights
    model.save(save_directory)

    #return model
    return model

  @staticmethod
  def finetune(save_directory, features, labels, verbose=1,
               learningRate=1e-5, 
               batch_size=10, 
               epochs=10):
    X = features
    y = labels

    #create 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    
    model = load_model(save_directory)
  
    model.compile(optimizer=Adam(learningRate),  # Very low learning rate
    loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

    model.summary()

    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)
    loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    print("(finetuning) Testing accuracy from within X_fine and y_fine: ", acc)
  
    return model

Pretraining Hyperparameters and Train/Test

In [117]:
#pretrain hyperparameters
batch_size = 100
epochs = 1
verbose = 1
# filters = (64, 100)
filters = (64, 100)
resLayers = 1
convLayers = 1
learningRate = 0.001
dropout = 0.8
kernel_size = 1


#save_directory is the path where the model will be saved after pretraining 
#and the path where the finetuning model will read from
save_directory = '/content/drive/My Drive/Machine_Learning/Saved Models/base.h5' 


#load data (specific to google drive)
X_test = np.load('/content/drive/My Drive/Machine_Learning/Data/X_test.npy')
y_test = np.load('/content/drive/My Drive/Machine_Learning/Data/y_test.npy')

X = np.load('/content/drive/My Drive/Machine_Learning/Data/X_reference.npy')
y = np.load('/content/drive/My Drive/Machine_Learning/Data/y_reference.npy')

X_fine = np.load('/content/drive/My Drive/Machine_Learning/Data/X_finetune.npy')
y_fine = np.load('/content/drive/My Drive/Machine_Learning/Data/y_finetune.npy')

indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

pretrained_model = ResNet.pretrain(save_directory=save_directory, features=X, labels=y, verbose=verbose, batch_size=batch_size, epochs=epochs, filters=filters, resLayers=resLayers, convLayers=convLayers, learningRate=learningRate, dropout=dropout, kernel_size=kernel_size)

pre_loss, pre_acc = pretrained_model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
print("Testing loss, accuracy after pretraining from X_test and y_test:", pre_loss, pre_acc)


Resnet Block
*ConvGroup (None, 500, 100) 100.0
*Shortcut shape (None, 500, 100)
Model: "resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_38 (InputLayer)           [(None, 1000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_115 (Conv1D)             (None, 500, 64)      320         input_38[0][0]                   
__________________________________________________________________________________________________
batch_normalization_153 (BatchN (None, 500, 64)      256         conv1d_115[0][0]                 
__________________________________________________________________________________________________
activation_152 (Activation)     (None, 500, 64)      0           batch_normalization_153[0][0]    
_____________

Finetuning Hyperparameters and Train/Test

In [118]:
#finetune hyperparameters
ft_batch_size = 10
ft_epochs = 8
ft_learningRate = 0.0001


finetuned_model = ResNet.finetune(save_directory=save_directory, features=X_fine, labels=y_fine, epochs=ft_epochs, learningRate=ft_learningRate, batch_size=ft_batch_size)

loss, acc = finetuned_model.evaluate(X_test, y_test, batch_size=ft_batch_size, verbose=verbose)
print("Pretrained accuracy: ", pre_acc)
print("Fintuned accuracy: ", acc)

Model: "resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_38 (InputLayer)           [(None, 1000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_115 (Conv1D)             (None, 500, 64)      320         input_38[0][0]                   
__________________________________________________________________________________________________
batch_normalization_153 (BatchN (None, 500, 64)      256         conv1d_115[0][0]                 
__________________________________________________________________________________________________
activation_152 (Activation)     (None, 500, 64)      0           batch_normalization_153[0][0]    
_____________________________________________________________________________________________

Todo:
*   Create hyperparameter tuning framework
*   Separate/clean up mounting google drive and loading files
*   Add confusion matrix
*   Comment code
*   Add clinical tuning method 



